In [1]:
# Import pandas package  
import pandas as pd 
import numpy as np
import altair as at
from geopy.exc import GeocoderTimedOut
from geopy.geocoders import Nominatim

In [85]:
# Lectura y conversión de archivo .cvs a DataFrame de Pandas
df_logistics=pd.read_csv('/content/synergy_logistics_database.csv',index_col=0)
# Dentro del dataframe de df_logistics aseguramos que la columna de date sea de tipo datetime
df_logistics['date'] = pd.to_datetime(df_logistics['date'],format='%d/%m/%y')
df_logistics.head()

,direction,origin,destination,year,date,product,transport_mode,company_name,total_value
register_id,,,,,,,,,
1,Exports,Japan,China,2015,2015-01-31,Cars,Sea,Honda,33000000
2,Exports,Japan,China,2015,2015-02-01,Cars,Sea,Honda,16000000
3,Exports,Japan,China,2015,2015-02-02,Cars,Sea,Honda,29000000
4,Exports,Japan,China,2015,2015-02-03,Cars,Sea,Honda,14000000
5,Exports,Japan,China,2015,2015-02-04,Cars,Sea,Honda,17000000


In [93]:
contry__world = 194
precencia_paices=df_logistics.loc[:,['origin','destination']]

precencia_paices= pd.unique(precencia_paices.values.ravel())

precencia_paices_dic = { country : 0 for country in precencia_paices }

count_precencia_orig=df_logistics['origin']

count_precencia_des=df_logistics['destination']

precencia_paices = pd.concat([count_precencia_orig,count_precencia_des ]).value_counts().to_frame().reset_index()

precencia_paices = precencia_paices.rename(columns={'index':'Counrty',0:'Precencia'})

contry__world = ((precencia_paices.count()/contry__world)*100).to_frame().reset_index().rename(columns={'index':'Counrty',0:'% Precencia Mundial'})
contry__world
print(contry__world)
print(precencia_paices)

     Counrty  % Precencia Mundial
0    Counrty            18.041237
1  Precencia            18.041237
                 Counrty  Precencia
0                  China       3775
1                    USA       3386
2                 Mexico       2974
3                Germany       2843
4                  Japan       2774
5                 France       1981
6            South Korea       1710
7                 Canada       1507
8            Netherlands       1495
9                Belgium       1491
10        United Kingdom       1460
11                Brazil       1418
12             Singapore       1123
13                 Spain       1110
14                Russia       1103
15                 Italy       1070
16              Thailand       1032
17             Australia        965
18                 India        745
19  United Arab Emirates        635
20               Vietnam        515
21               Austria        503
22           Switzerland        443
23             Argentina        39

In [18]:
longitude = []
latitude = []

def findGeocode(city):
       
    try:
          
        geolocator = Nominatim(user_agent="synergy")
          
        return geolocator.geocode(city)
      
    except GeocoderTimedOut:
          
        return findGeocode(city) 
  
for i in (precencia_paices["Counrty"]):
      
    if findGeocode(i) != None:
           
        loc = findGeocode(i)
          
        latitude.append(loc.latitude)
        longitude.append(loc.longitude)
       
    else:
        latitude.append(np.nan)
        longitude.append(np.nan)

In [19]:
precencia_paices["Longitude"] = longitude
precencia_paices["Latitude"] = latitude
  
precencia_paices.columns.values

array(['Counrty', 'Precencia', 'Longitude', 'Latitude'], dtype=object)

In [27]:
import altair as alt
from vega_datasets import data

countries = alt.topo_feature(data.world_110m.url, 'countries')

background = alt.Chart(countries).mark_geoshape(
    fill='#6cbdb3',
    stroke='white'
).project(
    "equirectangular"
).properties(
    width=900,
    height=500
)

points = alt.Chart(precencia_paices).mark_circle().encode(
    longitude='Longitude:Q',
    latitude='Latitude:Q',
    size='Precencia',
    color=alt.Color("Counrty", legend=alt.Legend(title="City"),scale=alt.Scale(scheme='dark2')),
    tooltip=['Counrty', 'Precencia', 'Longitude', 'Latitude']
)
background + points

alt.LayerChart(...)

In [53]:
bars = alt.Chart(contry__world.head(1)).mark_bar().encode(
    x=alt.X('% Precencia Mundial'),
    
)

text = bars.mark_text(
    align='left',
    baseline='middle',
    dx=3,  
    color='green'
).encode(
    text = alt.Text(f'% Precencia Mundial:Q', format = ".3s")
)

(bars + text).properties(title="Presencia a nivel mundial")

alt.LayerChart(...)